In [1]:
# source: https://maikros.github.io/yahoo-finance-python/

import requests                  # [handles the http interactions](http://docs.python-requests.org/en/master/) 
from bs4 import BeautifulSoup    # beautiful soup handles the html to text conversion and more
import re                        # regular expressions are necessary for finding the crumb (more on crumbs later)
from datetime import datetime    # string to datetime object conversion
from time import mktime          # mktime transforms datetime objects to unix timestamps

def _get_crumbs_and_cookies(stock):
    """
    get crumb and cookies for historical data csv download from yahoo finance
    
    parameters: stock - short-handle identifier of the company 
    
    returns a tuple of header, crumb and cookie
    """
    
    url = 'https://finance.yahoo.com/quote/{}/history'.format(stock)
    with requests.session():
        header = {'Connection': 'keep-alive',
                   'Expires': '-1',
                   'Upgrade-Insecure-Requests': '1',
                   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) \
                   AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36'
                   }
        
        website = requests.get(url, headers=header)
        soup = BeautifulSoup(website.text, 'lxml')
        crumb = re.findall('"CrumbStore":{"crumb":"(.+?)"}', str(soup))

        return (header, crumb[0], website.cookies)
    
    
def convert_to_unix(date):
    """
    converts date to unix timestamp
    
    parameters: date - in format (dd-mm-yyyy)
    
    returns integer unix timestamp
    """
    datum = datetime.strptime(date, '%d-%m-%Y')
    
    return int(mktime(datum.timetuple()))


def load_csv_data(stock, interval='1d', day_begin='01-03-2018', day_end='28-03-2018'):
    """
    queries yahoo finance api to receive historical data in csv file format
    
    parameters: 
        stock - short-handle identifier of the company
        
        interval - 1d, 1wk, 1mo - daily, weekly monthly data
        
        day_begin - starting date for the historical data (format: dd-mm-yyyy)
        
        day_end - final date of the data (format: dd-mm-yyyy)
    
    returns a list of comma seperated value lines
    """
    day_begin_unix = convert_to_unix(day_begin)
    day_end_unix = convert_to_unix(day_end)
    
    header, crumb, cookies = _get_crumbs_and_cookies(stock)
    
    with requests.session():
        url = 'https://query1.finance.yahoo.com/v7/finance/download/' \
              '{stock}?period1={day_begin}&period2={day_end}&interval={interval}&events=history&crumb={crumb}' \
              .format(stock=stock, day_begin=day_begin_unix, day_end=day_end_unix, interval=interval, crumb=crumb)
                
        website = requests.get(url, headers=header, cookies=cookies)
       
        return website.text # old: website.text.split('\n')[:-1]

def text4csv(stock='AAPL', day_begin='01-03-2018', day_end='28-03-2019'):
    text4csv = load_csv_data(stock, day_begin=day_begin, day_end=day_end)
    f = open(stock + '.csv', "w") 
    f.write(text4csv)
    f.close()

DAY_BEGIN = '01-03-2014'
DAY_END   = '28-03-2019'

import pandas as pd
def chk_csv(stock):
    df = pd.read_csv(stock + '.csv')
    print(stock, "\n", df.head(5), "\n")
    
def download_stock(stock):
    text4csv(stock, day_begin=DAY_BEGIN, day_end=DAY_END)
    chk_csv(stock)

In [3]:
stocks = ['BRK-B','SPY']
for stock in stocks:
    download_stock(stock)

BRK-B 
          Date        Open        High         Low       Close   Adj Close  \
0  2014-03-03  116.150002  116.750000  114.879997  116.059998  116.059998   
1  2014-03-04  117.129997  118.589996  117.000000  118.570000  118.570000   
2  2014-03-05  118.379997  119.080002  118.139999  118.980003  118.980003   
3  2014-03-06  119.169998  121.440002  119.110001  121.199997  121.199997   
4  2014-03-07  121.839996  122.919998  121.400002  122.669998  122.669998   

    Volume  
0  4556900  
1  4830400  
2  3458000  
3  5337700  
4  4384600   

SPY 
          Date        Open        High         Low       Close   Adj Close  \
0  2014-03-03  184.649994  185.449997  183.750000  184.979996  166.793854   
1  2014-03-04  186.789993  187.979996  186.750000  187.580002  169.138214   
2  2014-03-05  187.740005  188.070007  187.449997  187.750000  169.291519   
3  2014-03-06  188.210007  188.610001  187.779999  188.179993  169.679260   
4  2014-03-07  188.960007  188.960007  187.429993  188.259

In [16]:
stocks = ['TEAM'
         ,'CRM'
         ,'BABA'
         ,'AMZN'
         ,'AAPL'
         ,'FB'
         ,'GOOGL'
         ,'MSFT'
         ,'AYX']

# later consider save stocks to csv: https://stackoverflow.com/questions/2084069/create-a-csv-file-with-values-from-a-python-list

for stock in stocks:
    chk_csv(stock)

TEAM 
          Date       Open       High        Low      Close  Adj Close    Volume
0  2015-12-09  21.000000  21.000000  21.000000  21.000000  21.000000         0
1  2015-12-10  27.670000  28.500000  26.500000  27.780001  27.780001  15007300
2  2015-12-11  27.850000  28.500000  27.480000  27.500000  27.500000   2286100
3  2015-12-14  27.500000  27.860001  26.110001  26.260000  26.260000   1223500
4  2015-12-15  26.389999  26.948999  26.299999  26.500000  26.500000   1086400 

CRM 
          Date       Open       High        Low      Close  Adj Close    Volume
0  2014-03-03  60.650002  61.709999  59.820000  61.490002  61.490002   9303400
1  2014-03-04  62.180000  63.369999  62.009998  63.070000  63.070000   7927200
2  2014-03-05  63.070000  63.750000  62.939999  63.680000  63.680000   5464400
3  2014-03-06  63.750000  63.869999  62.730000  63.139999  63.139999   3958800
4  2014-03-07  63.200001  63.360001  59.730000  60.740002  60.740002  11239300 

BABA 
          Date       Open    